In [4]:
import numpy as np
import sympy as sym

#Ejercicio 4

def columnas (A,n):
    vector=np.array([])
    if n<=np.shape(A)[1]:
        for i in range(0,np.shape(A)[0]):
            vector=np.append(vector,A[i][n])
    else:
        "La columna indicada sobrepasa el índice de columnas de la matriz"
    return vector


                   
def multiplicador_matrices (A,B):
    dim_A=np.shape(A)
    dim_B=np.shape(B)
    matriz_multi=np.zeros(shape= (dim_A[0],dim_B[1]))
    if dim_A[1]==dim_B[0]:
        for i in range(0,np.shape(A)[0]) :
            for j in range(0,np.shape(B)[1]):
                matriz_multi[i][j]=np.dot(A[i],columnas(B,j))
    else:
        return ("Las matrices no se pueden multiplicar, maestro")

    return matriz_multi





1.0000000000000002 [-0.40824815  0.40824897  0.81649631]


In [11]:
#Ejercicio 11

#Tenemos la matriz
A=np.array([[1,2,-1],[1,0,1],[4,-4,5]])
A_inv=np.linalg.inv(A)
random=np.random.rand(3)


def hallar_vector_propio (A,vector,i):
    normal_0=vector/np.linalg.norm(vector)
    for j in range (i):
        eigenvector=np.dot(A,normal_0)
        eigenvector_normal=eigenvector/np.linalg.norm(eigenvector)
        normal_0=eigenvector_normal
    return normal_0

vector=hallar_vector_propio (A_inv,random,20)
vector_t=np.transpose(vector)
eigenvalue=np.dot(vector,vector_t)
print("El valor propio menor es:" , eigenvalue, " y su vector es: " ,vector)



El valor propio menor es: 0.9999999999999998  y su vector es:  [-0.40824764  0.40825156  0.81649527]


In [23]:
#Ejercicio 12

#Definamos nuestro sistema
sistema=np.array([lambda x,y: np.log((x**2)+(y**2)) - np.sin(x*y)-np.log(2)-np.log(np.pi),
     lambda x,y: np.exp(x-y)+np.cos(x*y)])

def GetF(G,r):
    
    n = r.shape[0]
    
    v = np.zeros_like(r)
    
    for i in range(n):
        v[i] = G[i](r[0],r[1])
        
    return v


def GetJacobian(f,r,h=1e-6):
    
    n = r.shape[0]
    
    J = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            
            rf = r.copy()
            rb = r.copy()
            
            rf[j] = rf[j] + h
            rb[j] = rb[j] - h
            
            J[i,j] = ( f[i](rf[0],rf[1]) - f[i](rb[0],rb[1])  )/(2*h)
            
    
    return J

def NewtonRaphson(G,r,itmax=1000,error=1e-9):
    
    it = 0
    d = 1.
    dvector = []
    
    while d > error and it < itmax:
        
        # Vector actual
        rc = r
        
        F = GetF(G,rc)
        J = GetJacobian(G,rc)
        InvJ = np.linalg.inv(J)
        
        r = rc - np.dot(InvJ,F)
        
        diff = r - rc
        
        d = np.max( np.abs(diff) )
        
        dvector.append(d)
        print(dvector)
        
        it += 1
    
    
    return r,dvector



[[1068051.77568967 1068051.77568967]
 [ 197311.88725604 -977889.30638776]]
